In [ ]:
import jax.numpy as jnp

import larch as lx

In [ ]:
d = lx.examples.MTC(format='dataset')

In [ ]:
m = lx.Model(d)
from larch import PX, P, X

m.utility_co[2] = P("ASC_SR2")  + P("hhinc#2") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#3") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

m.utility_ca = PX("tottime") + PX("totcost")

m.availability_var = 'avail'
m.choice_ca_var = 'chose'

m.title = "MTC Example 1 (Simple MNL)"
r = m.maximize_loglike(quiet=True)


In [ ]:
groupid = np.repeat(
    np.arange(1300),
    np.random.default_rng(0).choice([3,5], size=1300)
)[:m.dataset.n_cases]

m.groupid = groupid

In [ ]:
# dataset = m.dataset.assign_coords(groupid=xr.DataArray(groupid, dims='caseid'))
# dataset

In [ ]:
# fdataset = fold_dataset(dataset)
# fdataset

In [ ]:
# m.dataset = fdataset

In [ ]:
m.pf

In [ ]:
ca = jnp.asarray(m.dataset['ca'])
co = jnp.asarray(m.dataset['co'])
ch = jnp.asarray(m.dataset['ch'])
av = jnp.asarray(m.dataset['av'])

params = jnp.asarray(m.pvals)


In [ ]:
m.dataset

In [ ]:
m.mix_parameter("totcost", "totcost_s")

In [ ]:
m.pf

In [ ]:
m.unmangle()

In [ ]:
z = np.asarray(m.pvals).copy()

In [ ]:
# m.dataset = fdataset

In [ ]:
m.jax_loglike(z)

In [ ]:
m.jax_loglike(z)

In [ ]:
m.jax_d_loglike(z) # 3.39s

In [ ]:
m.jax_d_loglike(z) # 194ms

In [ ]:
m.n_draws = 1000
m.unmangle()

In [ ]:
m.jax_d_loglike(z) # 6.39s

In [ ]:
m.jax_d_loglike(z) # 1.83s

In [ ]:
m.jax_loglike(z) # 1.44s

In [ ]:
m.jax_loglike(z) # 455ms

In [ ]:
m.groupid = None

In [ ]:
m.jax_loglike(z) # 2.0s

In [ ]:
m.jax_loglike(z) #  482ms

In [ ]:
m.groupid = groupid

In [ ]:
m.jax_loglike(z) #  2.24s

In [ ]:
m.dataset